In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/Users/pratappadhy/Documents/Projects/Python/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embeddings.embed_query("hello AI")

[-0.03338826075196266,
 0.03453980013728142,
 0.05947456881403923,
 0.05928614363074303,
 -0.06353536248207092,
 -0.06819581985473633,
 0.08823318779468536,
 0.03444082662463188,
 -0.03278521075844765,
 -0.015814946964383125,
 0.020981697365641594,
 -0.018340256065130234,
 -0.039832208305597305,
 -0.08047077059745789,
 -0.014469243586063385,
 0.0332648865878582,
 0.01425927598029375,
 -0.034049950540065765,
 -0.1429157555103302,
 -0.023083392530679703,
 -0.02138018049299717,
 0.0026335662696510553,
 -0.047292690724134445,
 -0.010752714239060879,
 -0.06866797059774399,
 0.031124990433454514,
 0.07594592869281769,
 0.0011282660998404026,
 0.011631971225142479,
 -0.036039188504219055,
 0.04483757168054581,
 0.018390730023384094,
 0.12672801315784454,
 -0.0013597437646239996,
 0.00820667389780283,
 0.06909967213869095,
 -0.08076359331607819,
 -0.05841311067342758,
 0.0537545271217823,
 0.02622753381729126,
 -0.006828575395047665,
 -0.05635837838053703,
 0.0032929631415754557,
 -0.072501830

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
documents=["what is a capital of USA?",
           "Who is a president of USA?",
           "Who is a prime minister of India?"]

In [8]:
my_query="Narendra modi is prime minister of india?"

In [9]:
document_embedding=embeddings.embed_documents(documents)

In [10]:
document_embedding

[[0.11998694390058517,
  -0.021302636712789536,
  -0.04288087412714958,
  0.06645582616329193,
  -0.0643523707985878,
  -0.04424866661429405,
  0.022408461198210716,
  -0.04987306892871857,
  -0.023437578231096268,
  -0.033972032368183136,
  -0.014047931879758835,
  -0.0606592521071434,
  -0.00390676548704505,
  -0.017782099545001984,
  -0.04797104001045227,
  -0.06668158620595932,
  0.004103229846805334,
  -0.013092718087136745,
  0.04439776390790939,
  0.02235071361064911,
  0.00945960357785225,
  -0.02056451328098774,
  -0.0003355780791025609,
  -0.00568583095446229,
  0.05558693781495094,
  0.025123244151473045,
  -0.0028171860612928867,
  0.008758973330259323,
  0.0032552608754485846,
  -0.015963466838002205,
  0.014263720251619816,
  -0.11220847815275192,
  0.08968561887741089,
  -0.031083719804883003,
  -0.024223826825618744,
  0.006152121350169182,
  0.08058709651231766,
  0.01824999786913395,
  0.05568314343690872,
  0.016702674329280853,
  0.015895981341600418,
  0.0003411494

In [11]:
query_embedding=embeddings.embed_query(my_query)

In [12]:
len(query_embedding)

384

In [13]:
cosine_similarity([query_embedding],document_embedding)

array([[0.1175667 , 0.34324566, 0.81413239]])

In [14]:
from sklearn.metrics.pairwise import euclidean_distances

In [15]:
euclidean_distances([query_embedding], document_embedding)

array([[1.32848281, 1.14608408, 0.60970093]])

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [16]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [17]:
index=faiss.IndexFlatL2(384)


In [18]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x104c02cd0> >

In [19]:
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [20]:
vector_store.add_texts(["AI is future","AI is powerful","Dogs are cute"])

['1ed9d63b-0407-47ec-a16f-8d9ae3901400',
 '7b2121b8-092f-4636-ad45-1538aad29b21',
 'e9786565-b6b2-44a4-9fdd-48efe378c2e3']

In [21]:
vector_store.index_to_docstore_id

{0: '1ed9d63b-0407-47ec-a16f-8d9ae3901400',
 1: '7b2121b8-092f-4636-ad45-1538aad29b21',
 2: 'e9786565-b6b2-44a4-9fdd-48efe378c2e3'}

In [22]:
results = vector_store.similarity_search("Tell me about AI", k=3)

In [23]:
results


[Document(id='7b2121b8-092f-4636-ad45-1538aad29b21', metadata={}, page_content='AI is powerful'),
 Document(id='1ed9d63b-0407-47ec-a16f-8d9ae3901400', metadata={}, page_content='AI is future'),
 Document(id='e9786565-b6b2-44a4-9fdd-48efe378c2e3', metadata={}, page_content='Dogs are cute')]

| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [40]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [41]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [42]:
vector_store.add_documents(documents=documents)

['3c2e8735-b073-4302-a453-1b9380d11fa0',
 '85ff1f63-eab1-486f-aa69-e56eb18a9131',
 'cf74777f-8ae8-4ff7-a580-6ed28438e2ca',
 '5243ea40-cd2a-43dc-bdb1-c22aa60b1f4a',
 '7b983241-fe97-4663-bf87-4810f43f7945',
 '5d700df9-217e-4e7b-82e7-78d92570c966',
 '46e82003-7e1b-4f35-9acb-c417742b7317',
 'a793f400-d7de-49b8-b97c-877c0151a48a',
 'c121e7c9-61cd-4a7e-bac5-3511dc1c21b9',
 '6f7cea2f-72f1-4eef-bc8e-7518a676026e']

In [43]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2 #hyperparameter
    
)

[Document(id='cf74777f-8ae8-4ff7-a580-6ed28438e2ca', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='a793f400-d7de-49b8-b97c-877c0151a48a', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [44]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":{"$eq": "tweet"}}
    
)

[Document(id='cf74777f-8ae8-4ff7-a580-6ed28438e2ca', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='a793f400-d7de-49b8-b97c-877c0151a48a', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='6f7cea2f-72f1-4eef-bc8e-7518a676026e', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='3c2e8735-b073-4302-a453-1b9380d11fa0', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [45]:
result=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"news"}
    
)

In [47]:
result[0].metadata

{'source': 'news'}

In [48]:
result[0].page_content

'Robbers broke into the city bank and stole $1 million in cash.'

In [49]:
retriever=vector_store.as_retriever(search_kwargs={"k": 3})

In [50]:
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(id='cf74777f-8ae8-4ff7-a580-6ed28438e2ca', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='a793f400-d7de-49b8-b97c-877c0151a48a', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='6f7cea2f-72f1-4eef-bc8e-7518a676026e', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]

In [ ]:
inmemory(server)
ondisk(server)
cloud(yet to discuss)

In [51]:
vector_store.save_local("today's class faiss index")

In [52]:
new_vector_store=FAISS.load_local(
  "today's class faiss index",embeddings ,allow_dangerous_deserialization=True
)

In [53]:
new_vector_store.similarity_search("langchain")

[Document(id='cf74777f-8ae8-4ff7-a580-6ed28438e2ca', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='a793f400-d7de-49b8-b97c-877c0151a48a', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='7b983241-fe97-4663-bf87-4810f43f7945', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='46e82003-7e1b-4f35-9acb-c417742b7317', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [54]:
from langchain_community.document_loaders import PyPDFLoader

In [57]:
FILE_PATH=r"/Users/pratappadhy/Documents/Projects/Python/Pratap_Agentic_AI/2-LangChain-Basics/2.6-Vector-Database/2.6.1-FAISS/llama2.pdf"

In [58]:
loader=PyPDFLoader(FILE_PATH)

In [59]:
len(loader.load())

77

In [60]:
pages=loader.load()

In [61]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [62]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [63]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,#hyperparameter
    chunk_overlap=50 #hyperparemeter
)

In [64]:
split_docs = splitter.split_documents(pages)

In [65]:
len(split_docs)

615

In [66]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [67]:
vector_store.add_documents(documents=split_docs)

['2549ebf1-e46f-4522-ab4a-83547299bf23',
 '569ffde0-253d-4873-9ba3-5882afc4498d',
 '4267da2f-9a73-4a28-82b4-0570c40cf360',
 'f08991aa-48ad-4dd4-ac48-733ea2f302d3',
 '8d96f316-246f-4e49-b596-8bac83268ee9',
 '05883780-0a78-43f6-b6ab-75a0b0630a46',
 'ba5fcf76-d031-4175-bc7d-6c239380401c',
 'eaa8ef52-2709-45c0-b190-462cfc3cf034',
 '0028e78a-c332-4f2e-a9df-f914b469b805',
 '1a69e7f1-c5e0-4200-b61a-9250fc9865bd',
 '8a44688b-5186-47aa-b25e-d24e6fc7384a',
 '6bb0742f-868d-4f80-8293-797670de8e1f',
 '43789b9e-3cf2-4e50-beef-f228804088fd',
 '9918197c-476f-4a2b-a54a-4fdb3a88f80b',
 '51a2a2a6-9e66-4bf5-b8c8-22bdf2818fd9',
 'b5d62731-910f-4a97-9652-932ac8ecc93f',
 'f8056f8a-5a24-4edf-b1c2-4a17bc047767',
 '5b5111ed-c334-4f5b-9a62-806caa700c43',
 '2b7f98f6-71eb-40b4-bb07-526f6a5ad85f',
 '424f4d4e-04cd-485e-8799-cc8739c6f46c',
 'bbc82da4-963c-4782-adb6-4b086a89f81a',
 '8deb59a5-1cc1-4371-a2d5-292bdd23a01e',
 'aae212c3-e936-4b7a-801b-d445a9cb5e42',
 '7d2c2f1d-009b-4c4e-adfb-60b1dbb9144a',
 '573ff8b1-472f-

In [78]:
retriever=vector_store.as_retriever(
    search_kwargs={"k": 10} #hyperparameter
)

In [79]:
retriever.invoke("what is llama model?")

[Document(id='978f81cd-4673-4b91-b8f2-a60d4de1550e', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Users\\sunny\\agenticai-2.0\\data\\llama2.pdf', 'total_pages': 77, 'page': 3, 'page_label': '4'}, page_content='work (Section 6), and conclusions (Section 7).\n‡https://ai.meta.com/resources/models-and-libraries/llama/\n§We are delaying the release of the 34B model due to a lack of time to sufficiently red team.\n¶https://ai.meta.com/llama\n‖https://github.com/facebookresearch/llama\n4'),
 Document(id='89693474-8fe4-4edd-b381-6c2cec1a2a87', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywor

In [80]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [81]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

c:\Users\sunny\agenticai-2.0\env\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [82]:
import pprint


In [83]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [84]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
context(retriever),prompt(hub),model(google),parser(langchain)

In [86]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

In [87]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [88]:
rag_chain.invoke("what is llama model?")

'Llama is a large language model developed by Meta.  It has various versions, including Llama 1 and Llama 2, with different parameter sizes.  Llama 2 is designed for commercial and research use and is available under a custom commercial license.'